In [1]:
import os
import numpy as np
import cv2
import time
from resizeimage import resizeimage
import json
#from pyimagesearch.shapedetector import ShapeDetector
import argparse
import imutils

In [2]:
#os.chdir("C:/Users/Housna/Desktop/vid_proj")

In [3]:
os.chdir('C:/Users/Housna/Desktop/TEST_LSP')

In [4]:
# Ball Detection in a frame from .mp4
def detect_yellow_mp4(image):
    image = image[50:410, 0:640]
    original = image.copy()
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower = np.array([20, 30, 140], dtype="uint8")
    upper = np.array([45, 255, 255], dtype="uint8")
    mask = cv2.inRange(image, lower, upper)
    res = []
    x = 0
    y = 0
    k = "la balle n est pas sur le terrain"
    cnts = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
     
    for c in cnts:
        approx = cv2.approxPolyDP(c,0.001*cv2.arcLength(c,True),True)
        x = approx.ravel()[0]
        y = approx.ravel()[1]
        if len(approx) > 20 :
            #cv2.drawContours(original, [approx],0,(35,255,12),2)
            x,y,w,h = cv2.boundingRect(c)
            if len(approx) > 60 :
                cv2.rectangle(original, (x, y), (x + w, y + h), (36,255,12), 2)
                k = "La balle est en arrêt ou sous le pied d'un joueur"                
            else :
                cv2.rectangle(original, (x, y), (x + w, y + h), (36,255,12), 2)
                k = "la balle est en jeu"
    cv2.imshow('mask', mask)
    cv2.imshow('original', original)
    cv2.waitKey(100)
    cv2.destroyAllWindows()
    res.append(k)
    res.append(x)
    res.append(y)
    return res
    
# Ball Detection in a frame from .mkv
def detect_yellow_mkv(image):
    image = image[0:450, 0:640]
    original = image.copy()
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower = np.array([20, 50, 100], dtype="uint8")
    upper = np.array([45, 255, 255], dtype="uint8")
    mask = cv2.inRange(image, lower, upper)
    res = []
    x = 0
    y = 0
    k = "la balle n est pas sur le terrain"
    cnts = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
     
    for c in cnts:
        approx = cv2.approxPolyDP(c,0.001*cv2.arcLength(c,True),True)
        x = approx.ravel()[0]
        y = approx.ravel()[1]
        if len(approx) > 5 :
            #cv2.drawContours(original, [approx],0,(35,255,12),2)
            x,y,w,h = cv2.boundingRect(c)
            if len(approx) > 30 : 
                cv2.rectangle(original, (x, y), (x + w, y + h), (36,255,12), 2)
                k = "La balle est sous le pied d'un joueur"                
            else :
                cv2.rectangle(original, (x, y), (x + w, y + h), (36,255,12), 2)
                k = "la balle est en jeu"
    cv2.imshow('mask', mask)
    cv2.imshow('original', original)
    cv2.waitKey(100)
    cv2.destroyAllWindows()
    res.append(k)
    res.append(x)
    res.append(y)
    return res

#Detect a ball : returns a dict of the frame and the state of the ball
def ball_detect(frame): 
    rslt = {}
    rslt["frame"] = frame
    if 'mp4' in  frame :
        im = cv2.imread(frame,1)
        rslt["Ball state"] = detect_yellow_mp4(im)[0]
        rslt["Ball position"] = (detect_yellow_mp4(im)[1], detect_yellow_mp4(im)[2])

    elif 'mkv' in frame :
        im = cv2.imread(frame,1)
        rslt["Ball state"] = detect_yellow_mkv(im)[0]
        rslt["Ball position"] = (detect_yellow_mkv(im)[1], detect_yellow_mkv(im)[2])
    return rslt

#Creates a .json with the results of all the path's frames
def result_ball_detect(path):
    with open('results.txt', 'w') as outfile:
        for element in os.listdir(path):
             if element.endswith('.jpg') :
                        json.dump(ball_detect(element), outfile, indent=2)

In [ ]:
#ball_detect("2019_12_11__17_52_23.mkv__frame21354.jpg")
#ball_detect("191125_241563_7.mp4__frame997.jpg")
result_ball_detect('.')